In [1]:
#!pip install citipy

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import scipy.stats as st
import hvplot.pandas

# Impor the OpenWeatherMap API key
from api_keys import weather_api_key

In [4]:
state_data=pd.read_csv("final_data.csv")
state_data=state_data.loc[:,["State.1","Healthcare Score","Cost of Living Index","Homicide Rate (per 100,000)"]]
state_data=state_data.rename(columns ={"State.1": "State"})

state_data.head()

,State,Healthcare Score,Cost of Living Index,"Homicide Rate (per 100,000)"
0,California,56.74,134.5,0.313473
1,Hawaii,78.81,179.0,0.416218
2,New York,32.75,125.1,0.625129
3,New Hampshire,30.47,115.0,1.007925
4,Vermont,56.22,114.9,1.239215


In [5]:
#First, the URL of Open Weather Map  API
url= "http://api.openweathermap.org/data/2.5/weather?"
units="metric"

info_geo_state=[]

# Print to logger
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count=1
# Log the url, record, and set numbers

for state in state_data["State"]:
    #print("Processing Record %s of Set %s | %s" % (record_count,set_count, state))
    #this, is to see the name of the information that i need.
    state_url = f"{url}appid={weather_api_key}&units={units}&q="
    #url_city=city_url + "invercargill"
    #url_city
    try:
        state_weather = requests.get(state_url + state+",US").json()
        state_lat=state_weather["coord"]["lat"]
        state_lng=state_weather["coord"]["lon"]

        info_geo_state.append({"State":state,
                            "Lat":state_lat,
                            "Lng":state_lng})
    except:
        print("City not found. Skipping...")
        pass
    #city_weather["coord"]["lat"]
    record_count += 1 
info_geo_state

Beginning Data Retrieval     
-----------------------------


[{'State': 'California', 'Lat': 38.3004, 'Lng': -76.5074},
 {'State': 'Hawaii', 'Lat': 20.7503, 'Lng': -156.5003},
 {'State': 'New York', 'Lat': 40.7143, 'Lng': -74.006},
 {'State': 'New Hampshire', 'Lat': 43.667, 'Lng': -71.4998},
 {'State': 'Vermont', 'Lat': 44.0003, 'Lng': -72.7498},
 {'State': 'Maine', 'Lat': 45.5003, 'Lng': -69.2498},
 {'State': 'New Jersey', 'Lat': 40.1671, 'Lng': -74.4999},
 {'State': 'North Dakota', 'Lat': 47.5003, 'Lng': -100.0007},
 {'State': 'Massachusetts', 'Lat': 42.3657, 'Lng': -71.1083},
 {'State': 'Idaho', 'Lat': 44.5002, 'Lng': -114.2512},
 {'State': 'Iowa', 'Lat': 42.0003, 'Lng': -93.5005},
 {'State': 'Alaska', 'Lat': 64.0003, 'Lng': -150.0003},
 {'State': 'Utah', 'Lat': 39.2502, 'Lng': -111.751},
 {'State': 'Arizona', 'Lat': 34.5003, 'Lng': -111.501},
 {'State': 'Montana', 'Lat': 47.0003, 'Lng': -109.751},
 {'State': 'South Dakota', 'Lat': 44.5003, 'Lng': -100.2507},
 {'State': 'Wyoming', 'Lat': 43.0002, 'Lng': -107.5009},
 {'State': 'Kansas', 'Lat':

In [6]:
state_weather = requests.get(state_url + state+",US").json()
state_weather

{'coord': {'lon': -77.0003, 'lat': 38.9171},
 'weather': [{'id': 802,
   'main': 'Clouds',
   'description': 'scattered clouds',
   'icon': '03d'}],
 'base': 'stations',
 'main': {'temp': 25.72,
  'feels_like': 25.81,
  'temp_min': 23.49,
  'temp_max': 27.65,
  'pressure': 1022,
  'humidity': 56},
 'visibility': 10000,
 'wind': {'speed': 1.54, 'deg': 170},
 'clouds': {'all': 40},
 'dt': 1690989949,
 'sys': {'type': 2,
  'id': 2078663,
  'country': 'US',
  'sunrise': 1690970971,
  'sunset': 1691021935},
 'timezone': -14400,
 'id': 4138106,
 'name': 'District of Columbia',
 'cod': 200}

In [7]:
state_data_df=pd.DataFrame(info_geo_state)
state_data_df.head()

,State,Lat,Lng
0,California,38.3004,-76.5074
1,Hawaii,20.7503,-156.5003
2,New York,40.7143,-74.0060
3,New Hampshire,43.6670,-71.4998
4,Vermont,44.0003,-72.7498


In [8]:
state_data_complete=pd.merge(state_data_df,state_data,on="State",how="outer")
state_data_complete.head()

,State,Lat,Lng,Healthcare Score,Cost of Living Index,"Homicide Rate (per 100,000)"
0,California,38.3004,-76.5074,56.74,134.5,0.313473
1,Hawaii,20.7503,-156.5003,78.81,179.0,0.416218
2,New York,40.7143,-74.0060,32.75,125.1,0.625129
3,New Hampshire,43.6670,-71.4998,30.47,115.0,1.007925
4,Vermont,44.0003,-72.7498,56.22,114.9,1.239215


In [11]:
states_map=state_data_complete.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width = 800,
    frame_height = 600,
    size="Total Homicide",
    scale=10,
    color="State"
)

states_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (State)

In [16]:
priorities=[1/3,1/3,1/3]

state_data_complete["Rank Same Priority"]=state_data_complete["Cost of Living Index"]*(1/3)+state_data_complete["Healthcare Score"]*(1/3)+state_data_complete["Homicide Rate (per 100,000)"]*(1/3)
state_data_complete.head()

,State,Lat,Lng,Healthcare Score,Cost of Living Index,"Homicide Rate (per 100,000)",Rank Same Priority
0,California,38.3004,-76.5074,56.74,134.5,0.313473,63.851158
1,Hawaii,20.7503,-156.5003,78.81,179.0,0.416218,86.075406
2,New York,40.7143,-74.0060,32.75,125.1,0.625129,52.825043
3,New Hampshire,43.6670,-71.4998,30.47,115.0,1.007925,48.825975
4,Vermont,44.0003,-72.7498,56.22,114.9,1.239215,57.453072


In [17]:
state_data_complete["Rank Healthcare Priority"]=state_data_complete["Cost of Living Index"]*(1/4)+state_data_complete["Healthcare Score"]*(1/2)+state_data_complete["Homicide Rate (per 100,000)"]*(1/4)
state_data_complete["Rank Cost Living Priority"]=state_data_complete["Cost of Living Index"]*(1/2)+state_data_complete["Healthcare Score"]*(1/4)+state_data_complete["Homicide Rate (per 100,000)"]*(1/4)
state_data_complete["Rank Homicide Priority"]=state_data_complete["Cost of Living Index"]*(1/4)+state_data_complete["Healthcare Score"]*(1/4)+state_data_complete["Homicide Rate (per 100,000)"]*(1/2)
state_data_complete.head()

,State,Lat,Lng,Healthcare Score,Cost of Living Index,"Homicide Rate (per 100,000)",Rank Same Priority,Rank Healthcare Priority,Rank Cost Living Priority,Rank Homicide Priority
0,California,38.3004,-76.5074,56.74,134.5,0.313473,63.851158,62.073368,81.513368,47.966736
1,Hawaii,20.7503,-156.5003,78.81,179.0,0.416218,86.075406,84.259054,109.306554,64.660609
2,New York,40.7143,-74.0060,32.75,125.1,0.625129,52.825043,47.806282,70.893782,39.775064
3,New Hampshire,43.6670,-71.4998,30.47,115.0,1.007925,48.825975,44.236981,65.369481,36.871463
4,Vermont,44.0003,-72.7498,56.22,114.9,1.239215,57.453072,57.144804,71.814804,43.399607


## PLOTTING PRIORITIES
### SAME PRIORITY OF EVERY SCORE

In [28]:
states_map=state_data_complete.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width = 800,
    frame_height = 600,
    size="Rank Same Priority",
    scale=1,
    color="State"
)

states_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (State,Rank Same Priority)

In [25]:
states_map=state_data_complete.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width = 800,
    frame_height = 600,
    size="Rank Healthcare Priority",
    scale=2,
    color="State"
)

states_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (State,Rank Healthcare Priority)

In [23]:
states_map=state_data_complete.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width = 800,
    frame_height = 600,
    size="Rank Cost Living Priority",
    scale=3,
    color="State"
)

states_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (State,Rank Cost Living Priority)

In [26]:
states_map=state_data_complete.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width = 800,
    frame_height = 600,
    size="Rank Homicide Priority",
    scale=3,
    color="State"
)

states_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (State,Rank Homicide Priority)